**NGS Analysis Practice**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
tools_dir = "/content/drive/MyDrive/ngs_practice/Tools"
data_dir = "/content/drive/MyDrive/ngs_practice/Data"

In [ ]:
!wget -O "{tools_dir}/fastqc.zip" https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
!unzip "{tools_dir}/fastqc.zip" -d "{tools_dir}"
!chmod +x "{tools_dir}/FastQC/fastqc"
!"{tools_dir}/FastQC/fastqc" -help

In [ ]:
!mkdir "{tools_dir}/fastp"
!wget -O "{tools_dir}/fastp/fastp" http://opengene.org/fastp/fastp
!chmod a+x "{tools_dir}/fastp/fastp"
!"{tools_dir}/fastp/fastp"

In [ ]:
!wget -O - "http://sourceforge.net/projects/bio-bwa/files/bwakit/bwakit-0.7.12_x64-linux.tar.bz2/download" \
| tar -xj -C "{tools_dir}"
!"{tools_dir}/bwa.kit/run-bwamem"

In [ ]:
!wget -O "{tools_dir}/samtools-new.tar.bz2" https://github.com/samtools/samtools/releases/download/1.22/samtools-1.22.tar.bz2
!tar -xvjf "{tools_dir}/samtools-new.tar.bz2" -C "{tools_dir}"
%cd "{tools_dir}/samtools-1.22"
!./configure --prefix="{tools_dir}/samtools-1.22"
!make
!make install
%cd
!"{tools_dir}/samtools-1.22/bin/samtools"

In [ ]:
!apt-get update
!apt-get install openjdk-17-jdk -y
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
!java -version


In [ ]:
!wget -O "{tools_dir}/picard.jar" https://github.com/broadinstitute/picard/releases/download/3.4.0/picard.jar
!chmod +x "{tools_dir}/picard.jar"
!java -jar "{tools_dir}/picard.jar"

In [ ]:
!wget -O "{tools_dir}/gatk.zip" https://github.com/broadinstitute/gatk/releases/download/4.5.0.0/gatk-4.5.0.0.zip
!unzip "{tools_dir}/gatk.zip" -d "{tools_dir}"
!chmod +x "{tools_dir}/gatk-4.5.0.0/gatk"
!"{tools_dir}/gatk-4.5.0.0/gatk" --list

In [ ]:
!"{tools_dir}/FastQC/fastqc" "{data_dir}/sample/LSP4_279g_S19_L001_R1_001.fastq.gz" "{data_dir}/sample/LSP4_279g_S19_L001_R2_001.fastq.gz" -o "{data_dir}/outfile"

In [ ]:
!"{tools_dir}/fastp/fastp" -i "{data_dir}/sample/LSP4_279g_S19_L001_R1_001.fastq.gz" -I "{data_dir}/sample/LSP4_279g_S19_L001_R2_001.fastq.gz" -o "{data_dir}/sample/normal_R1.fastq.gz" -O "{data_dir}/sample/normal_R2.fastq.gz" -q 20 -u 30 -n 10 -l 50 -h "{data_dir}/outfile/report_fastp.html"

In [ ]:
!"{tools_dir}/bwa.kit/bwa" mem -t 2 -R '@RG\tID:L7VN7_1\tSM:sample19\tPL:ILLUMINA\tLB:lib1\tPU:L7VN7_1' "{data_dir}/ref/Homo_sapiens_assembly38.fasta" "{data_dir}/sample/normal_R1.fastq.gz" "{data_dir}/sample/normal_R2.fastq.gz" | "{tools_dir}/samtools-1.22/bin/samtools" view -Shb -o "{data_dir}/outfile/normal.bam"

In [ ]:
!"{tools_dir}/samtools-1.22/bin/samtools" sort -o "{data_dir}/outfile/normal-sorted.bam" "{data_dir}/outfile/normal.bam"
!"{tools_dir}/samtools-1.22/bin/samtools" index "{data_dir}/outfile/normal-sorted.bam"


In [ ]:
!java -jar "{tools_dir}/picard.jar" MarkDuplicates INPUT="{data_dir}/outfile/normal-sorted.bam" METRICS_FILE="{data_dir}/outfile/normal-metrics.txt" OUTPUT="{data_dir}/outfile/normal-dedup.bam" CREATE_INDEX=true

In [ ]:
!"{tools_dir}/gatk-4.5.0.0/gatk" BaseRecalibrator -R "{data_dir}/ref/Homo_sapiens_assembly38.fasta" -I "{data_dir}/outfile/normal-dedup.bam" --known-sites "{data_dir}/ref/Homo_sapiens_assembly38.known_indels.vcf.gz" -O "{data_dir}/outfile/normal.grp"
!"{tools_dir}/gatk-4.5.0.0/gatk" ApplyBQSR -R "{data_dir}/ref/Homo_sapiens_assembly38.fasta" -I "{data_dir}/outfile/normal-dedup.bam" --bqsr-recal-file "{data_dir}/outfile/normal.grp" -O "{data_dir}/outfile/normal-bqsr.bam"